In [1]:
import cv2
import numpy as nps
import matplotlib.pyplot as plt

Base Code (cleaned up version)

In [6]:
def initialize_point():
    #point is initialized by clicking with the mouse
    ix, iy = -1, -1 #note that these values are just random numbers

    def onMouse(event, x, y, flags, params):
        nonlocal ix, iy
        if event == cv2.EVENT_LBUTTONDOWN:
            ix, iy = x, y
            
    
    cv2.namedWindow("Camera")
    cv2.setMouseCallback("Camera", onMouse)

    cap = cv2.VideoCapture(0)

    #turn off camera with esc button
    while True:
        _, frame = cap.read()
        

        #display coordinates on the camera frame
        if ix != -1 and iy != -1:
            cv2.putText(frame, f"Coordinates: ({ix}, {iy})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 0, 225), 2)
            cv2.circle(frame, (ix, iy), 5, (0, 0, 225), -1)

        cv2.imshow("Camera", frame)
        
        if cv2.waitKey(1) == 27:
            #old_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            break
    
    cap.release()
    cv2.destroyAllWindows()

    return ix, iy

if __name__ == "__main__":
    initial_x, initial_y = initialize_point()
    print("Selected Point Coordinates: ", initial_x, initial_y)

Selected Point Coordinates:  200 200


Integration of Lucas-Kanade Optical Flow Tracking using recorded Videos